In [2]:
import pandas as pd
import os

In [8]:
michelin_restaurants_path = 'data/cleaned/michelin_cleaned.csv'
yelp_restaurants_path = 'data/cleaned/yelp_cleaned.csv'

In [9]:
michelin_df = pd.read_csv(michelin_restaurants_path)
yelp_df = pd.read_csv(yelp_restaurants_path)

In [ ]:
michelin_df
yelp_df

In [12]:
michelin_df = michelin_df[['Name', 'Address', 'Award']]

michelin_df.to_csv("data/cleaned/michelin_table.csv", index=False)

In [16]:
yelp_df = yelp_df[yelp_df['is_open'] == 1]

yelp_df_1 = yelp_df[['business_id','name','address','city','state','postal_code',
                    'latitude','longitude','stars','review_count','categories']]
yelp_df_1.to_csv("data/cleaned/yelp_table.csv", index=False)